# Import thư viện

In [1]:
import numpy as np
import os
import cv2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from convGRU import ConvGRU
import torch
from torch.utils.data import DataLoader, TensorDataset
from Yogi_torch import Yogi
import matplotlib.pyplot as plt
import keras
import tensorflow_addons as tfa
from deap import base, creator, tools, algorithms
import random
import h5py
from torch import nn

c:\Users\ductr\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\ductr\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounte

# Dữ liệu đầu vào

### Đọc dữ liệu ảnh

In [2]:
# Đường dẫn đến thư mục chứa ảnh
image_folder = r"E:\BacSon\Fair_2024_ConvLSTM_ConvGRU\data"

# Đọc từng file ảnh
image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

In [3]:
# Chuyển ảnh về ma trận ảnh Gray

def matrix_images(image_folder, image_files):
    matrix_img = []
    for img_file in image_files:
        img_path = os.path.join(image_folder, img_file)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        matrix_img.append(image)

    return matrix_img

data = matrix_images(image_folder, image_files)

### Chuẩn hóa dữ liệu

In [4]:
# Áp dụng Min-Max Scaling chuẩn hóa dữ liệu
def min_max_scaling(data):
    # Chuyển danh sách các mảng numpy thành một mảng numpy đa chiều
    data_array = np.array(data)

    # Tính giá trị min và max trên toàn bộ dữ liệu
    data_min = np.min(data_array)
    data_max = np.max(data_array)

    # Áp dụng công thức Min-Max Scaling
    scaled_data = (data_array - data_min) / (data_max - data_min)

    return scaled_data

In [5]:
data = min_max_scaling(data)
data = np.expand_dims(data, axis=1)

In [6]:
def create_image_sequences(data, time_steps):
    num_samples, height, width, channels = data.shape
    num_frames = num_samples - time_steps
    input_sequences = np.zeros((num_frames, time_steps, height, width, channels))
    labels = np.zeros((num_frames, height, width, channels))  # Khởi tạo mảng label

    for i in range(num_frames):
        input_sequences[i] = data[i:i+time_steps]
        labels[i] = data[i+time_steps]  # Xác định label là frame tiếp theo

    return input_sequences, labels

# Tạo chuỗi dữ liệu với 4 ảnh liên tiếp từ dữ liệu hình ảnh
time_step = 4
X_data, y_data = create_image_sequences(data, time_step)

### Chia dữ liệu

In [7]:
# Chia dữ liệu
size = int(len(X_data) * 0.8)
size_val = int((len(X_data) - size) / 2)

X_train = X_data[:size]
X_val = X_data[size:size + size_val]
X_test = X_data[-size_val:]

y_train = y_data[:size]
y_val = y_data[size:size + size_val]
y_test = y_data[- size_val:]

In [8]:
# Kiểm tra kích thước của X_train
print("Kích thước của X_train:", X_train.shape)
print("Kích thước của X_val:", X_val.shape)
print("Kích thước của X_test:", X_test.shape)

print("Kích thước của y_train:", y_train.shape)
print("Kích thước của y_val:", y_val.shape)
print("Kích thước của y_test:", y_test.shape)

Kích thước của X_train: (48, 4, 1, 150, 150)
Kích thước của X_val: (6, 4, 1, 150, 150)
Kích thước của X_test: (6, 4, 1, 150, 150)
Kích thước của y_train: (48, 1, 150, 150)
Kích thước của y_val: (6, 1, 150, 150)
Kích thước của y_test: (6, 1, 150, 150)


# Mô hình GA-ConvGRU

### Tinh chỉnh các thông số đầu vào cho mô hình

In [9]:
# Chuyển đổi các mảng thành các tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Tạo DataLoader cho training and validation
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_dataloader = DataLoader(test_dataset)

### Câu trúc mô hình

In [10]:
def conv_gru(X_train, hidden_dim, kernel_size, activation_function):
    # Các tham số đầu vào
    input_size = (X_train.shape[-2:])
    channels = X_train.shape[2]
    num_layers = 3

    # Xây dựng mô hình
    model_GRU = ConvGRU(input_size=input_size, 
                        input_dim=channels,
                        hidden_dim=hidden_dim, 
                        kernel_size=kernel_size, 
                        num_layers=num_layers,
                        dtype=torch.FloatTensor,
                        batch_first=True,
                        bias=True,
                        return_all_layers=False,
                        activation=activation_function)
    
    return model_GRU

### Áp dụng GA

In [11]:
import torch.nn as nn

def save_model_to_hdf5(model, file_path):
    with h5py.File(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.create_dataset(name, data=param.data.cpu().numpy())

# Hàm mục tiêu cho mô hình
def fitness_function(X_train, train_dataloader, val_dataloader, hidden_dim1, hidden_dim2, kernel_size1, kernel_size2, 
                     kernel_size3, activation_function, optimizer, num_epochs=1, save_path_model = ""):
    # Gọi mô hình
    hidden_dim = [hidden_dim1, hidden_dim2, 1]
    kernel_size = [kernel_size1, kernel_size2, kernel_size3]
    model = conv_gru(X_train, hidden_dim, kernel_size, activation_function)

    criterion = nn.MSELoss()
    best_val_loss = float('inf')
    train_loss_epoch = []
    val_loss_epoch = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for X_batch, y_batch in train_dataloader:
            optimizer.zero_grad()
            outputs, _ = model(X_batch)
            output_last_layer = outputs[-1]
            loss = criterion(output_last_layer, y_batch)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        
        avg_train_loss = total_loss / len(train_dataloader)
        train_loss_epoch.append(avg_train_loss)
        
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_dataloader:
                outputs, _ = model(X_batch)
                output_last_layer = outputs[-1]
                val_loss = criterion(output_last_layer, y_batch)
                total_val_loss += val_loss.item()
        
        avg_val_loss = total_val_loss / len(val_dataloader)
        val_loss_epoch.append(avg_val_loss)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            
    # lưu lại model tốt nhất
    if save_path_model:
        os.makedirs(os.path.dirname(save_path_model), exist_ok=True)
        save_model_to_hdf5(model, save_path_model)
        history_model = {"loss" : train_loss_epoch, "val_loss" : val_loss_epoch}
        
        return best_val_loss, model, history_model
    else:
        return best_val_loss, model

In [13]:
# Hàm đánh giá cá thể trong quần thể
def evaluate(individual):
    hidden_dim1, hidden_dim2, kernel_size1, kernel_size2, kernel_size3, activation = individual
    num_epochs = 50

    print("Train Adam")
    optimizer_Adam = torch.optim.Adam(conv_gru(X_train, 
                                               [hidden_dim1, hidden_dim2, 1], 
                                               [kernel_size1, kernel_size2, kernel_size3], activation).parameters(), 
                                               lr = 1e-3)
    val_loss_adam, _ = fitness_function(X_train, train_dataloader, val_dataloader, 
                                        hidden_dim1, hidden_dim2, 
                                        kernel_size1, kernel_size2, kernel_size3, activation,
                                        optimizer_Adam, num_epochs=num_epochs)
    return val_loss_adam,

In [14]:
# Định nghĩa hàm tạo quần thể
def create_population(n, hidden_dim1, hidden_dim2, kernel_size_func1, kernel_size_func2, kernel_size_func3, activation):
    population = [creator.Individual([hidden_dim1(), hidden_dim2(), kernel_size_func1(), kernel_size_func2(), 
                                      kernel_size_func3(), activation()]) for _ in range(n)]
    return population

In [15]:
# Hàm đột biến cá thể
def mutate_individual(individual, indpb):
    if random.random() < indpb:
        individual[0] = random.choice([16, 32, 64, 128])  # filters1
    if random.random() < indpb:
        individual[1] = random.choice([16, 32, 64, 128])  # filters2
    if random.random() < indpb:
        individual[2] = random.choice([(3, 3), (5, 5), (7, 7)])  # kernel_size1
    if random.random() < indpb:
        individual[3] = random.choice([(3, 3), (5, 5), (7, 7)])  # kernel_size2
    if random.random() < indpb:
        individual[4] = random.choice([(3, 3), (5, 5), (7, 7)])  # kernel_size3
    if random.random() < indpb:
        individual[5] = random.choice([nn.Tanh, nn.ReLU, nn.Sigmoid])  # activation
    return individual,

In [16]:
# Số lượng quần thể ban đầu
num_populations = 1
# Số lượng cá thể trong quần thể ban đầu
individuals = 2
# Quá trình lai ghép và đột biến
num_generations = 0

epochs = 2

In [17]:
# Khởi tạo hàm mục tiêu
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Khởi tạo toolbox
toolbox = base.Toolbox()
toolbox.register("attr_filters", random.choice, [16, 32, 64, 128])
toolbox.register("attr_kernel_size", random.choice, [(3, 3), (5, 5), (7, 7)])
toolbox.register("attr_activation", random.choice, [nn.Tanh, nn.ReLU, nn.Sigmoid])
# toolbox.register("attr_activation", random.choice, ["tanh", "relu", "sigmoid"])

# Đăng ký cá thể và quần thể
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (toolbox.attr_filters, toolbox.attr_filters, 
                  toolbox.attr_kernel_size, toolbox.attr_kernel_size, 
                  toolbox.attr_kernel_size, toolbox.attr_activation), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Đăng ký các toán tử lai ghép, đột biến và chọn lọc
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate_individual, indpb=0.15)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Tạo và in quần thể ban đầu
populations = []
for i in range(num_populations):
    population = create_population(individuals, toolbox.attr_filters, toolbox.attr_filters, 
                                   toolbox.attr_kernel_size, toolbox.attr_kernel_size, toolbox.attr_kernel_size, 
                                   toolbox.attr_activation)
    populations.append(population)
    print(f"Quần thể ban đầu {i + 1}:")
    for j, ind in enumerate(population):
        print(f"Cá thể thứ {j}: {ind}")
    print()

# Đánh giá fitness và chọn lọc cá thể tốt nhất cho mỗi quần thể ban đầu
best_individuals = []
for population in populations:
    fits = toolbox.map(toolbox.evaluate, population)
    for fit, ind in zip(fits, population):
        ind.fitness.values = fit
    best_ind = min(population, key=lambda ind: ind.fitness.values)
    best_individuals.append(best_ind)
    print(f"Cá thể tốt nhất trong quần thể thứ {populations.index(population) + 1}: {best_ind} ban đầu")
print()

# Quá trình lai ghép và đột biến
print("Bắt đầu quá trình lai ghép và đột biến...")
print()

for gen in range(num_generations):
    # Tạo quần thể con bằng cách lai ghép và đột biến từ các cá thể tốt nhất
    offspring = algorithms.varAnd(best_individuals, toolbox, cxpb=0.7, mutpb=0.15)
    print(f"Quần thể con F{gen + 1} đã được tiến hóa:")
    for i, ind in enumerate(offspring):
        print(f"Cá thể thứ {i + 1}: {ind}")

    # Đánh giá fitness của quần thể con
    fits = list(map(toolbox.evaluate, offspring))
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    best_ind = min(offspring, key=lambda ind: ind.fitness.values)
    best_individuals.append(best_ind)
    print(f"Cá thể tốt nhất trong quần thể con F{gen + 1}: {best_ind}")

    # Chọn lọc cá thể để tạo quần thể mới cho thế hệ tiếp theo
    best_individuals = toolbox.select(best_individuals, k=4)
    print()

# In ra cá thể tốt nhất cuối cùng
best_ind = min(best_individuals, key=lambda ind: ind.fitness.values)
print("Cá thể tốt nhất cuối cùng:")
print(best_ind)

Quần thể ban đầu 1:
Cá thể thứ 0: [64, 128, (7, 7), (7, 7), (3, 3), <class 'torch.nn.modules.activation.ReLU'>]
Cá thể thứ 1: [16, 64, (3, 3), (7, 7), (5, 5), <class 'torch.nn.modules.activation.Sigmoid'>]

Train Adam


c:\Users\ductr\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 1, 150, 150])) that is different to the input size (torch.Size([1, 4, 1, 150, 150])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/1, Train Loss: 0.04539739204725871, Val Loss: 0.04052261325220267
Train Adam
Epoch 1/1, Train Loss: 0.12163971721505125, Val Loss: 0.12038123980164528
Cá thể tốt nhất trong quần thể thứ 1: [64, 128, (7, 7), (7, 7), (3, 3), <class 'torch.nn.modules.activation.ReLU'>] ban đầu

Bắt đầu quá trình lai ghép và đột biến...

Cá thể tốt nhất cuối cùng:
[64, 128, (7, 7), (7, 7), (3, 3), <class 'torch.nn.modules.activation.ReLU'>]


# Đánh giá và test mô hình

### Biểu đồ train_loss và val_loss

In [18]:
def plot_loss_model_ConvGRU(history):
    # Trích xuất giá trị loss trên tập huấn luyện và kiểm tra
    train_loss = history['loss']
    val_loss = history['val_loss']

    # Vẽ biểu đồ
    plt.figure(figsize=(8, 5))
    plt.plot(train_loss, linestyle='-', label='Train loss')
    plt.plot(val_loss, linestyle='-', label='Val loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train loss và Val loss của mô hình')
    plt.yscale('log')  # Scale y-axis to logarithmic scale
    plt.legend()  # Thêm nhãn chú thích
    plt.show()

### Các tham số từ cá thể tốt nhất và training mô hình

In [19]:
hidden_dim1, hidden_dim2, kernel_size1, kernel_size2, kernel_size3, activation = best_ind

print(hidden_dim1, hidden_dim2, kernel_size1, kernel_size2, kernel_size3, activation)

64 128 (7, 7) (7, 7) (3, 3) <class 'torch.nn.modules.activation.ReLU'>


In [23]:
model_Adam = conv_gru(X_train, [hidden_dim1, hidden_dim2, 1], [kernel_size1, kernel_size2, kernel_size3], activation)
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=1e-3)
save_model_ConvGRU_Adam = './KetQua/ConvGRU/best_model_ConvGRU_Adam.hdf5'

val_loss_Adam, best_model_Adam, history_model_Adam = fitness_function(X_train, train_dataloader, val_dataloader, 
                                                                      hidden_dim1, hidden_dim2, 
                                                                      kernel_size1, kernel_size2, kernel_size3, 
                                                                      activation,
                                                                      optimizer_Adam, 
                                                                      num_epochs=epochs, 
                                                                      save_path_model = save_model_ConvGRU_Adam)

In [ ]:
plot_loss_model_ConvGRU(history_model_Adam)

In [24]:
model_Yogi = conv_gru(X_train, [hidden_dim1, hidden_dim2, 1], [kernel_size1, kernel_size2, kernel_size3], activation)
optimizer_Yogi = Yogi(model_Yogi.parameters(), lr=1e-3)
save_model_ConvGRU_Yogi = './KetQua/ConvGRU/best_model_ConvGRU_Yogi.hdf5'

val_loss_Yogi, best_model_Yogi, history_model_Yogi = fitness_function(X_train, train_dataloader, val_dataloader, 
                                                                      hidden_dim1, hidden_dim2, 
                                                                      kernel_size1, kernel_size2, kernel_size3, 
                                                                      activation,
                                                                      optimizer_Yogi, 
                                                                      num_epochs=epochs, 
                                                                      save_path_model = save_model_ConvGRU_Yogi)

In [ ]:
plot_loss_model_ConvGRU(history_model_Yogi)

### Đánh giá mô hình

In [21]:
def load_model_from_hdf5(file_path, model):
    with h5py.File(file_path, 'r') as f:
        for name, param in model.named_parameters():
            if name in f:
                param_data = torch.tensor(f[name][:])
                param.data.copy_(param_data)
            else:
                print(f"Không tìm thấy {name} trong file HDF5.")
    return model

# def loss_model(X_train_gru, best_model_path, val_dataloader):
def loss_model(X_train_gru, hidden_dim1, hidden_dim2, kernel_size1, kernel_size2, kernel_size3, 
               activation, model_path, val_dataloader):
    model_eval = conv_gru(X_train_gru, [hidden_dim1, hidden_dim2, 1], [kernel_size1, kernel_size2, kernel_size3], activation)
    model_eval = load_model_from_hdf5(model_path, model_eval)
    model_eval.eval()

    criterion = torch.nn.MSELoss()
    # Tính toán tổng loss trên tập validation
    total_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_dataloader:
            outputs, _ = model_eval(X_batch)
            output_last_layer = outputs[-1]
            val_loss = criterion(output_last_layer, y_batch)
            total_val_loss += val_loss.item()

    # Tính toán average validation loss
    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Average Validation Loss: {avg_val_loss}")

    return avg_val_loss

def Compare_Adam_Yogi_GRU(X_train_gru, hidden_dim1, hidden_dim2, kernel_size1, kernel_size2, kernel_size3, activation,
                          model_path_Adam, model_path_Yogi, val_dataloader):
    loss_Adam = loss_model(X_train_gru, hidden_dim1, hidden_dim2, kernel_size1, kernel_size2, kernel_size3, activation,
                           model_path_Adam, val_dataloader)
    loss_Yogi = loss_model(X_train_gru, hidden_dim1, hidden_dim2, kernel_size1, kernel_size2, kernel_size3, activation,
                           model_path_Yogi, val_dataloader)
    if loss_Adam < loss_Yogi:
        print("Best Model Adam")
        best_model_path = model_path_Adam
    else:
        print("Best Model YoGi")
        best_model_path = model_path_Yogi

    return best_model_path

In [25]:
best_model_path_ConvGRU = Compare_Adam_Yogi_GRU(X_train, 
                                                hidden_dim1, hidden_dim2, 
                                                kernel_size1, kernel_size2, kernel_size3, 
                                                activation,
                                                save_model_ConvGRU_Adam, save_model_ConvGRU_Yogi, 
                                                val_dataloader)

RuntimeError: The size of tensor a (7) must match the size of tensor b (3) at non-singleton dimension 3

### Dự đoán và đánh giá mô hình trên tập test

In [ ]:
def test_model_ConvGRU(X_train_gru, best_model_path, test_dataloader, y_test_gru):
    model_eval = conv_gru(X_train_gru, [hidden_dim1, hidden_dim2, 1], [kernel_size1, kernel_size2, kernel_size3], activation)
    model_eval = load_model_from_hdf5(best_model_path, model_eval)
    model_eval.eval()

    predictions = []
    with torch.no_grad():
        for X_batch, y_batch in test_dataloader:
            outputs, _ = model_eval(X_batch)
            output_last_image = outputs[-1][:, -1, ...]
            predictions.append(output_last_image.cpu().numpy())

    predicted_images = np.concatenate(predictions)
    predicted_images = np.squeeze(predicted_images)

    mse = mean_squared_error(y_test_gru.flatten(), predicted_images.flatten())
    print("Mean Squared Error (MSE):", mse)
    mae = mean_absolute_error(y_test_gru.flatten(), predicted_images.flatten())
    print("Mean Absolute Error (MAE):", mae)
    r2 = r2_score(y_test_gru.flatten(), predicted_images.flatten())
    print("R-squared (R2) Score:", r2)

    fig, axs = plt.subplots(1, min(len(predicted_images), 5), figsize=(15, 25))
    for i in range(min(len(predicted_images), 5)):
        axs[i].imshow(predicted_images[i])
        axs[i].set_title(f"Ảnh dự đoán {i+1}")
    plt.show()

    return predicted_images

In [ ]:
# Dự đoán trên model tốt nhất
test_model_ConvGRU(X_train, best_model_path_ConvGRU, test_dataloader, y_test)